In [4]:
%pip install -U monai


Note: you may need to restart the kernel to use updated packages.


In [9]:
import os
import monai
from monai.transforms import (
    Compose,
    LoadImaged,
    EnsureChannelFirstd,
    ScaleIntensityd,
    ToTensord,
)
from monai.data import CacheDataset, DataLoader
from monai.networks.nets import UNet
from monai.networks.layers import Norm
from monai.losses import DiceLoss
from monai.metrics import DiceMetric
from monai.optimizers import Novograd
from monai.inferers import sliding_window_inference
import torch

# Set up the directory paths for images and labels
base_dir = "./datasets/data/datasets_unet/"
image_dir_train = os.path.join(base_dir, "train/images")
label_dir_train = os.path.join(base_dir, "train/masks")

# Get the list of image and label files
images_train = sorted([os.path.join(image_dir_train, f) for f in os.listdir(image_dir_train) if f.endswith('.png')])
labels_train = sorted([os.path.join(label_dir_train, f) for f in os.listdir(label_dir_train) if f.endswith('.png')])

# Create a list of dictionaries for paired image and label paths
data_dicts_train = [{'image': image_name, 'label': label_name} for image_name, label_name in zip(images_train, labels_train)]

# Define the transformations to preprocess the images
train_transforms = Compose([
    LoadImaged(keys=['image', 'label']),
    EnsureChannelFirstd(keys=['image', 'label']),
    ScaleIntensityd(keys=['image', 'label']),
    ToTensord(keys=['image', 'label']),
])

# Create a MONAI dataset and a data loader for training
train_ds = CacheDataset(data=data_dicts_train, transform=train_transforms, cache_rate=1.0)
train_loader = DataLoader(train_ds, batch_size=2, shuffle=True)

# Set up the U-Net model
model = UNet(
    dimensions=2,
    in_channels=1,
    out_channels=1, # If your dataset has binary segmentation, you need only 1 output channel
    channels=(16, 32, 64, 128, 256),
    strides=(2, 2, 2, 2),
    num_res_units=2,
    norm=Norm.BATCH,
)

# Set up the loss function and optimizer
loss_function = DiceLoss(to_onehot_y=True, sigmoid=True)
optimizer = Novograd(model.parameters(), lr=0.001)

# Start the training loop (simplified version)
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    for batch_data in train_loader:
        inputs, labels = batch_data['image'].cuda(), batch_data['label'].cuda()
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    print(f"epoch {epoch + 1}/{num_epochs}, average loss: {epoch_loss / len(train_loader)}")

# Note: For actual training, you also need to include validation, checkpoint saving,
# and use metrics to evaluate the model's performance.


FileNotFoundError: [WinError 3] 系统找不到指定的路径。: './datasets/data/datasets_unet/train/images'